# Erstellen Energiebedarszeitreihe der Ursprungsdaten zum Validieren der Simulation

In [16]:
import pandas as pd
import numpy as np 
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Globalparameter

In [17]:
# Tagtyp: 1 = Werktag, 2 = Samstag, 3 = Sonntag
TYPE_D = 1 
# Breite der Zeitschritte
T_STEP = 15 
# Ladeszenario: 1 = nur Zuhause, 2 = Zuhause und Arbeit, 3 = Überall
CHARGE_SCEN = 3
# Ladeleistung 
PCHARGE_SLOW = 3.7 
# Abspeichern der simulierten Fahrzeuge
simulated_evs = []

In [18]:
class Electric_Vehicle(object):
    model_df = pd.read_excel(r"C:\Users\thoma\Desktop\ev-modelling-repo\Rohdaten\EV_Modelle_Tabelle.xlsx", index_col='Modell')
    SEGMENTS = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -9])
    # Dummy Wahrscheinlichkeiten aus MOP Studie
    PROB_SEGMENT = [0.0592, #1
        0.1958, #2
        0.2621, #3
        0.1597, #4
        0.029, #5
        0.0058, #6
        0.0354, #7
        0.0122, #8
        0.0637, #9
        0.0586, #10
        0.0386, #11
        0.0058, #12
        0.0528, #13
        0.0213 #-9
        ]

    
    def __init__(self):
        segment = self.choose_segment()
        model = self.choose_model(segment)
        self.MODEL = model
        
        # Bug da Modelle mehrfach in Liste keine eindeutige Zuordnung sondern Array als output -> fixed -> elegantere Lösung?
        # Segment des Fahrzeugs
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Segment"]):
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"]
        else:
            self.SEGMENT = Electric_Vehicle.model_df.at[model, "Segment"][0]
        
        # Batteriekapazität in kWh
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Batterie"]):
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"]
        else:
            self.CAPACITY = Electric_Vehicle.model_df.at[model, "Batterie"][0]
        
        # Verbrauch in kWh/100km
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Verbrauch"]):
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"]
        else:
            self.CONSUMPTION = Electric_Vehicle.model_df.at[model, "Verbrauch"][0]
        
        # Leistung -> aktuell nicht gebraucht
        if np.isscalar(Electric_Vehicle.model_df.at[model, "Leistung"]):
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"]
        else:
            self.POWER = Electric_Vehicle.model_df.at[model, "Leistung"][0]
            
        self.SOC = 100
        self.trip_no = 0
        self.trips = []
    
    def choose_segment(self): 
        # wähle p zufällig auf Basis gegebener W'keiten
        choice = np.random.choice(Electric_Vehicle.SEGMENTS, p=Electric_Vehicle.PROB_SEGMENT)
        
        # Vorerst Wahl des häufigsten Semgents bei keiner Angabe -> später überarbeiten
        if choice == -9: 
            choice = 3
        
        return choice
    
    def choose_model(self, segment):
        models = Electric_Vehicle.model_df
        
        # filtern der infragekommenden Fahrzeuge über Segment
        filt = models["Segment"] == segment
        choices = models[filt]
        
        # Wahl einse zufälligen Fahrzeugs aus der Liste
        pick = np.random.randint(0, len(choices))
        model = choices.index[pick]
        return model
    
    # drive() und charge() testen
    
    def drive(self, distance):
        trip_consumption = distance * (self.CONSUMPTION / 100) 
        self.SOC = self.SOC - (trip_consumption / self.CAPACITY) * 100
                
    
    def check_distance(self, distance):
        # überprüfe SOC für gegebene Distanz ausreichend ist
        trip_consumption = distance * (self.CONSUMPTION / 100)
        if trip_consumption < self.SOC/100 * self.CAPACITY:
            return True 
        else:
            return False
        
    def max_distance(self):
        # maximale Distanz die mit aktuellem SOC zurückgelegt werden kann
        max_dist = round(((self.SOC/100)*self.CAPACITY / self.CONSUMPTION) * 100, 2)
        return max_dist
        
    def charge(self):       
        # Fahrzeug befindet sich im Zielzustand des letzten Trips
        trip = self.trips[len(self.trips)-1]
        # nötige Zeit zum vollständigen Aufladen in Minuten
        t_charge_full = (((100 - ev.SOC)/100 * ev.CAPACITY) / PCHARGE_SLOW) * 60
        # Ladezeit beschränkt durch Zeit zum vollständigen Aufladen und Länge des Aufenthalts
        t_charge = min(t_charge_full, trip.stay_duration)
        # update SOC über Ladedauer und Ladeleistung
        self.SOC = self.SOC + ((t_charge/60 * PCHARGE_SLOW) / self.CAPACITY) * 100
        trip.charge_start = int(trip.arrival)
        trip.charge_end = int(round(trip.charge_start + t_charge))

In [19]:
class Trip(object):
    
    def __init__(self, whyfrom, departure):
        self.trip_id = None
        self.trip_no = None
        self.whyfrom = whyfrom
        self.whyto = None
        self.departure = departure
        self.departure_t = round(float(departure/T_STEP))
        self.arrival = None
        self.trip_duration = None
        self.distance = None
        self.stay_duration = None
        self.SOC_start = None
        self.SOC_end = None
        self.charge_start = None 
        self.charge_end = None

In [20]:
# einlesen des bearbeiteten Trips-Datensatz (mit errechneten Aufenhtalten)
# bereits auf Wochentage gefiltert
df = pd.read_csv(r'C:\Users\thoma\Desktop\ev-modelling-repo\Rohdaten\NHTS_trips_processed_WT_Aufenthalt.csv')

In [21]:
df.head()

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t,Stay_duration
0,300000071,1,3,1,1,4,8.439,15,1000,1015,600,615,40,295
1,300000071,1,3,2,4,1,8.286,20,1510,1530,910,930,61,1110
2,300000073,1,3,1,1,2,3.621,15,845,900,525,540,35,330
3,300000073,1,3,2,2,1,3.610,15,1430,1445,870,885,58,1080
4,300000081,4,6,1,1,2,12.902,15,1115,1130,675,690,45,720


In [22]:
df = df.sort_values(by=["ID", "Trip_no"])

In [23]:
df.tail(20)

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t,Stay_duration
596260,407943012,1,6,3,4,2,4.683000,12,1138,1150,698,710,47,280
596261,407943012,1,6,4,2,3,10.828000,16,1630,1646,990,1006,66,36
596262,407943012,1,6,5,3,1,7.689000,12,1722,1734,1042,1054,69,866
596263,407943013,1,5,1,1,5,14.669000,26,1631,1657,991,1017,66,1
596264,407943013,1,5,2,5,3,1.257000,7,1658,1705,1018,1025,68,53
596265,407943013,1,5,3,3,5,1.257000,8,1758,1806,1078,1086,72,3
596266,407943013,1,5,4,5,1,15.389000,29,1809,1838,1089,1118,73,1313
596267,407943016,1,5,1,1,4,17.015175,26,1631,1657,991,1017,66,72
596268,407943016,1,5,2,4,1,18.978464,29,1809,1838,1089,1118,73,1313
596269,3046754510,2,5,1,1,5,6.544298,10,1400,1410,840,850,56,5


In [24]:
df.shape

(596280, 14)

## Analoge Simulation der EVs / SOCs und Ladevorgänge zur Fahrtensimulation auf Basis originaler Trips

In [25]:
total_evs = []
#rows = df_test.shape[0] 
ev = None
for row in range(df.shape[0]):
    if (df.at[row, "Trip_no"]) == 1 or (df.at[row, "ID"] != df.at[row-1, "ID"]):
      
        if ev:
            if ev.SOC < 100:
                location = ev.trips[len(ev.trips)-1].whyto
                # obere Schranke der Ladezeit, tatsächliche wird in ev.charge() bestimmt
                duration = ev.trips[len(ev.trips)-1].stay_duration
                if CHARGE_SCEN == 1:
                    # geladen wird nur wenn sich das Fahrzeug sich Zuhause befindet und sich dort länger als 15 Minuten aufhält
                    if location == 1 and duration > 15:
                        ev.charge()

                elif CHARGE_SCEN == 2:
                    # geladen wird nur wenn sich das Fahrzeug Zuhause oder auf der Arbeit befindet und sich dort länger als 15 Minuten aufhält
                    if (location == 1 or location == 2) and duration > 15:
                        ev.charge()

                elif CHARGE_SCEN == 3:
                    # geladen wird in jedem Zustand, sofern die Parkdauer 15 Minuten übersteigt
                    if duration > 15:
                        ev.charge()   
            total_evs.append(ev)
            
        ev = Electric_Vehicle()
    trip = Trip(df.at[row, "Whyfrom"], df.at[row, "Departure"])
    trip.trip_id = df.at[row, "ID"]
    trip.trip_no = df.at[row, "Trip_no"]
    trip.whyto = df.at[row, "Whyto"]
    trip.arrival = df.at[row, "Arrival"]
    trip.trip_duration = df.at[row, "Trip_duration"]
    trip.distance = df.at[row, "Distance"]
    trip.stay_duration = df.at[row, "Stay_duration"]
    
    if ev.SOC < 100:
        location = ev.trips[len(ev.trips)-1].whyto
        # obere Schranke der Ladezeit, tatsächliche wird in ev.charge() bestimmt
        duration = ev.trips[len(ev.trips)-1].stay_duration
        if CHARGE_SCEN == 1:
            # geladen wird nur wenn sich das Fahrzeug sich Zuhause befindet und sich dort länger als 15 Minuten aufhält
            if location == 1 and duration > 15:
                ev.charge()

        elif CHARGE_SCEN == 2:
            # geladen wird nur wenn sich das Fahrzeug Zuhause oder auf der Arbeit befindet und sich dort länger als 15 Minuten aufhält
            if (location == 1 or location == 2) and duration > 15:
                ev.charge()

        elif CHARGE_SCEN == 3:
            # geladen wird in jedem Zustand, sofern die Parkdauer 15 Minuten übersteigt
            if duration > 15:
                ev.charge()    
            
    trip.SOC_start = round(ev.SOC, 1)
    # Fahrvorgang -> update SOC 
    ev.drive(trip.distance)
    trip.SOC_end = round(ev.SOC, 1)
    ev.trips.append(trip)
    


In [26]:
from collections import defaultdict

# erzeugen Dictionary "Key : List"
total_trips_dict = defaultdict(list)

# speichern der Trips jedes einzelnen Fahrzeugs im dict
for ev in total_evs:
    for trip in ev.trips:
        # .__dict__.items() returns Dictionary mit allen Member Variablen und dazugehörigen Werten des Objekts
        for key, val in trip.__dict__.items():
            total_trips_dict[key].append(val)

# umwandeln in DataFrame
final_data = pd.DataFrame(total_trips_dict)

In [27]:
final_data.shape

(596274, 14)

In [28]:
# Spaltennamen umformatieren
final_data = final_data.rename(str.capitalize, axis='columns')
final_data.head()

,Trip_id,Trip_no,Whyfrom,Whyto,Departure,Departure_t,Arrival,Trip_duration,Distance,Stay_duration,Soc_start,Soc_end,Charge_start,Charge_end
0,300000071,1,1,4,600,40,615,15,8.439,295,100.0,95.9,615.0,642.0
1,300000071,2,4,1,910,61,930,20,8.286,1110,100.0,96.0,930.0,956.0
2,300000073,1,1,2,525,35,540,15,3.621,330,100.0,97.3,540.0,548.0
3,300000073,2,2,1,870,58,885,15,3.610,1080,100.0,97.3,885.0,893.0
4,300000081,1,1,2,675,45,690,15,12.902,720,100.0,96.4,690.0,720.0


## Warum 6 Trips weniger?

In [29]:
import pickle

path = r"C:\Users\thoma\Desktop\ev-modelling-repo\Simulationsauswertung\EBZ NHTS\Werktag\Auswertungsergebnisse\NHTS_Trips_Werktag_CS3_Ladezeiten.pickle"

pickle.dump(final_data, open(path,"wb"))

In [30]:
df.sort_values(by=["Trip_no"], ascending=False)

,ID,Type_day,TRPTRANS,Trip_no,Whyfrom,Whyto,Distance,Trip_duration,Departure_hhmm,Arrival_hhmm,Departure,Arrival,Departure_t,Stay_duration
270928,304679941,5,3,50,4,1,11.375000,19,2355,14,1435,14,96,1126
270927,304679941,5,3,49,1,4,12.250000,15,2021,2036,1221,1236,81,199
270926,304679941,5,3,47,2,2,5.852000,12,1922,1934,1162,1174,77,47
263473,304555151,2,6,47,2,1,4.514000,13,1825,1838,1105,1118,74,742
270925,304679941,5,3,46,2,2,13.559000,11,1910,1921,1150,1161,77,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327058,400142303,3,4,1,1,3,9.053000,15,1700,1715,1020,1035,68,75
327060,400142304,3,4,1,1,2,13.088596,20,615,635,375,395,25,640
327062,400142812,4,4,1,1,4,6.396000,15,815,830,495,510,33,421
327066,400142813,4,3,1,1,2,20.633000,30,715,745,435,465,29,615
